# Embeddings
https://platform.openai.com/docs/models#embeddings

임베딩(Embeddings)은 텍스트를 수치적으로 표현한 값으로, 두 텍스트 간의 연관성을 측정하는 데 사용된다.

임베딩은 검색, 군집화(clustering), 추천 시스템, 이상 탐지, 분류와 같은 작업에 유용하다.

**모델 및 출력 차원**

| 모델 이름                     | 설명                                                              | 출력 차원 |
|-------------------------------|-------------------------------------------------------------------|-----------|
| **text-embedding-3-large**   | 영어 및 비영어 작업 모두에서 가장 강력한 성능을 가진 모델           | 3,072     |
| **text-embedding-3-small**   | 2세대 ada 임베딩 모델보다 성능이 향상된 모델                        | 1,536     |
| **text-embedding-ada-002**   | 1세대 모델 16개를 대체하는 가장 강력한 2세대 임베딩 모델             | 1,536     |


## MTEB Leaderboard
**Massive Text Embedding Benchmark (MTEB) Leaderboard**

https://huggingface.co/spaces/mteb/leaderboard

**MTEB Leaderboard**는 Hugging Face에서 제공하는 벤치마크 리더보드 페이지로, 다양한 언어 모델(Language Model)과 임베딩 모델(Embedding Model)의 성능을 객관적으로 비교·평가하는 공간이다.

**MTEB Leaderboard에서 순위 산정 방식**

**MTEB Leaderboard**의 순위는 다양한 자연어 처리 태스크(분류, 클러스터링, 검색, 문장 유사도 등)에서 모델이 얻은 점수들의 평균을 기반으로 산정된다. 즉, 여러 벤치마크 데이터셋에서 모델의 성능을 측정하고, 이를 종합하여 평균 점수를 계산한 뒤, 이 평균 점수가 높은 순서대로 모델이 정렬된다.

**주요 평가 방식**

- **평가 태스크 종류**
  - 분류(Classification): F1 점수
  - 클러스터링(Clustering): V-measure
  - 쌍 분류(Pair Classification): Average Precision
  - 재정렬(Reranking): MRR@k, MAP
  - 검색(Retrieval): nDCG@k
  - 의미 유사도(STS): Spearman correlation
  - 요약(Summarization): Spearman correlation  
  각 태스크별로 대표적인 평가 지표가 다르며, 모델은 여러 태스크에서 평가를 받는다[2].

- **평균 점수 산정**
  - 각 태스크별로 모델이 얻은 점수를 모두 합산한 뒤, 태스크 수로 나누어 평균 점수를 구한다.
  - 이 평균 점수가 리더보드의 기본 순위 기준이 된다.

- **부분 평가 가능**
  - 모든 태스크를 수행하지 않아도 특정 태스크만 평가받아 부분 리더보드에 오를 수 있다. 예를 들어, 클러스터링 태스크만 평가받아 해당 부분 순위에 표시될 수 있다.




In [18]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [19]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

text = '안녕하세요~'

response = client.embeddings.create(
    model='text-embedding-3-small',
    input=[text]
)
print(response)

print(len(response.data[0].embedding))

CreateEmbeddingResponse(data=[Embedding(embedding=[0.006884642411023378, -0.08130010217428207, -0.006427524611353874, 0.021306157112121582, 0.03779585286974907, -0.03752826899290085, -0.0452212318778038, 0.035788990557193756, -0.0462469607591629, -0.029411640018224716, -0.05409600958228111, 0.003882715478539467, 0.012230693362653255, 0.00863507017493248, -0.03732758387923241, 0.02918865531682968, -0.044217802584171295, 0.01925470121204853, -0.027672361582517624, 0.019444238394498825, 0.061454493552446365, 0.01404801569879055, -0.010703249834477901, -0.019957100972533226, 0.038598597049713135, 0.05579069256782532, 0.05681641772389412, -0.027605466544628143, 0.0445522777736187, -0.05128640681505203, 0.027114899829030037, -0.038665492087602615, -0.014427089132368565, -0.053025685250759125, -0.027360182255506516, 0.004186531528830528, 0.01599912904202938, -0.022354183718562126, -0.02515263669192791, -0.037706658244132996, -0.014505133964121342, -0.00541294552385807, -0.004041591659188271, 

In [20]:
def texts_to_embedding(texts, model='text-embedding-3-small'):
    # 특수문자/개행문자등을 제거하면 임베딩품질 높아진다.
    texts = [text.replace('\n', ' ') for text in texts]
    response = client.embeddings.create(
        model=model,
        input=texts
    )
    return [data.embedding for data in response.data]

output = texts_to_embedding(['hello world', 'byebye world'])

import numpy as np
np.array(output).shape

(2, 1536)

## 음식리뷰 유사도 검색

In [21]:
!gdown 1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD

Downloading...
From: https://drive.google.com/uc?id=1tSQZQFYD64_mrL9CjDcn6KruZp7_smuD
To: /content/fine_food_reviews_1k.csv
100% 439k/439k [00:00<00:00, 108MB/s]


In [22]:
# 데이터로드
import pandas as pd

df = pd.read_csv('fine_food_reviews_1k.csv', index_col=0)
df.head()

Time   ProductId          UserId  Score  \
Unnamed: 0                                                  
0           1351123200  B003XPF9BO  A3R7JR3FMEBXQB      5   
1           1351123200  B003JK537S  A3JBPC3WFUT5ZP      1   
2           1351123200  B000JMBE7M   AQX1N6A51QOKG      4   
3           1351123200  B004AHGBX4  A2UY46X0OSNVUQ      3   
4           1351123200  B001BORBHO  A1AFOYZ9HSM2CZ      5   

                                                      Summary  \
Unnamed: 0                                                      
0           where does one  start...and stop... with a tre...   
1                                           Arrived in pieces   
2                   It isn't blanc mange, but isn't bad . . .   
3                 These also have SALT and it's not sea salt.   
4                                      Happy with the product   

                                                         Text  n_tokens  
Unnamed: 0                                                               
0           Wanted to save some to bring to my Chicago fam...        33  
1           Not pleased at all. When I opened the box, mos...        26  
2           I'm not sure that custard is really custard wi...       242  
3           I like the fact that you can see what you're g...       216  
4           My dog was suffering with itchy skin.  He had ...        76

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Time       1000 non-null   int64 
 1   ProductId  1000 non-null   object
 2   UserId     1000 non-null   object
 3   Score      1000 non-null   int64 
 4   Summary    1000 non-null   object
 5   Text       1000 non-null   object
 6   n_tokens   1000 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 62.5+ KB


In [24]:
# Summary + Text -> Combined
df['combined'] = df['Summary'].str.strip() + '; ' + df['Text'].str.strip()
df.head()

Time   ProductId          UserId  Score  \
Unnamed: 0                                                  
0           1351123200  B003XPF9BO  A3R7JR3FMEBXQB      5   
1           1351123200  B003JK537S  A3JBPC3WFUT5ZP      1   
2           1351123200  B000JMBE7M   AQX1N6A51QOKG      4   
3           1351123200  B004AHGBX4  A2UY46X0OSNVUQ      3   
4           1351123200  B001BORBHO  A1AFOYZ9HSM2CZ      5   

                                                      Summary  \
Unnamed: 0                                                      
0           where does one  start...and stop... with a tre...   
1                                           Arrived in pieces   
2                   It isn't blanc mange, but isn't bad . . .   
3                 These also have SALT and it's not sea salt.   
4                                      Happy with the product   

                                                         Text  n_tokens  \
Unnamed: 0                                                                
0           Wanted to save some to bring to my Chicago fam...        33   
1           Not pleased at all. When I opened the box, mos...        26   
2           I'm not sure that custard is really custard wi...       242   
3           I like the fact that you can see what you're g...       216   
4           My dog was suffering with itchy skin.  He had ...        76   

                                                     combined  
Unnamed: 0                                                     
0           where does one  start...and stop... with a tre...  
1           Arrived in pieces; Not pleased at all. When I ...  
2           It isn't blanc mange, but isn't bad . . .; I'm...  
3           These also have SALT and it's not sea salt.; I...  
4           Happy with the product; My dog was suffering w...

In [25]:
# 임베딩변환
df['embedding'] = texts_to_embedding(df['combined'].tolist())
df.head()

Time   ProductId          UserId  Score  \
Unnamed: 0                                                  
0           1351123200  B003XPF9BO  A3R7JR3FMEBXQB      5   
1           1351123200  B003JK537S  A3JBPC3WFUT5ZP      1   
2           1351123200  B000JMBE7M   AQX1N6A51QOKG      4   
3           1351123200  B004AHGBX4  A2UY46X0OSNVUQ      3   
4           1351123200  B001BORBHO  A1AFOYZ9HSM2CZ      5   

                                                      Summary  \
Unnamed: 0                                                      
0           where does one  start...and stop... with a tre...   
1                                           Arrived in pieces   
2                   It isn't blanc mange, but isn't bad . . .   
3                 These also have SALT and it's not sea salt.   
4                                      Happy with the product   

                                                         Text  n_tokens  \
Unnamed: 0                                                                
0           Wanted to save some to bring to my Chicago fam...        33   
1           Not pleased at all. When I opened the box, mos...        26   
2           I'm not sure that custard is really custard wi...       242   
3           I like the fact that you can see what you're g...       216   
4           My dog was suffering with itchy skin.  He had ...        76   

                                                     combined  \
Unnamed: 0                                                      
0           where does one  start...and stop... with a tre...   
1           Arrived in pieces; Not pleased at all. When I ...   
2           It isn't blanc mange, but isn't bad . . .; I'm...   
3           These also have SALT and it's not sea salt.; I...   
4           Happy with the product; My dog was suffering w...   

                                                    embedding  
Unnamed: 0                                                     
0           [0.030276387929916382, -0.020651785656809807, ...  
1           [0.01129516027867794, 0.03488067910075188, -0....  
2           [0.0022991024889051914, 0.004840383306145668, ...  
3           [-0.015768831595778465, 0.013766037300229073, ...  
4           [7.225894660223275e-05, -0.06748031824827194, ...

In [26]:
df['embedding'].tolist()

In [27]:
# embed_df로 변환
embed_df = df[['embedding']]
# embed_df = df['embedding'].to_frame()
embed_df.index = df['combined']
embed_df.head()

embedding
combined                                                                                             
where does one  start...and stop... with a trea...  [0.030276387929916382, -0.020651785656809807, ...
Arrived in pieces; Not pleased at all. When I o...  [0.01129516027867794, 0.03488067910075188, -0....
It isn't blanc mange, but isn't bad . . .; I'm ...  [0.0022991024889051914, 0.004840383306145668, ...
These also have SALT and it's not sea salt.; I ...  [-0.015768831595778465, 0.013766037300229073, ...
Happy with the product; My dog was suffering wi...  [7.225894660223275e-05, -0.06748031824827194, ...

In [28]:
# 유사도 검색
from sklearn.metrics.pairwise import cosine_similarity

def review_search(query, emb_df, top_n=5):
    # 검색어 동일한 임베딩변환
    query_emb = texts_to_embedding([query])

    embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])
    top_n_texts = embed_df.sort_values('cos_sim', ascending=False).head(top_n)
    return top_n_texts

review_search('market', embed_df)

/tmp/ipython-input-28-4287974552.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  embed_df['cos_sim'] = embed_df['embedding'].apply(lambda emb: cosine_similarity([emb], query_emb)[0, 0])


embedding  \
combined                                                                                                
awesome; I WILL BUY IT IF U LOWER UR PRICE TO E...  [0.006983668077737093, -0.02385917864739895, -...   
Metromint or Bust!; Chilled Oragemint is a clas...  [-0.006673663388937712, -0.020904269069433212,...   
Metromint or Bust!; Chilled Oragemint is a clas...  [-0.006673663388937712, -0.020904269069433212,...   
expectations met; Is what it says it is would n...  [0.03926253691315651, -0.0065216985531151295, ...   
Value Value Value; Dogs love product and this m...  [0.006547991652041674, -0.023272063583135605, ...   

                                                     cos_sim  
combined                                                      
awesome; I WILL BUY IT IF U LOWER UR PRICE TO E...  0.265445  
Metromint or Bust!; Chilled Oragemint is a clas...  0.263337  
Metromint or Bust!; Chilled Oragemint is a clas...  0.263337  
expectations met; Is what it says it is would n...  0.244889  
Value Value Value; Dogs love product and this m...  0.220926